In [ ]:
import pandas as pd
import numpy as np
import re
import csv
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import torch

np.random.seed(500)

In [ ]:
df_prescriptions = pd.read_csv('C:/Users/harim/OneDrive/Documents/GitHub/dl4h_final_project_1/data/processed/ndc_codes_extracted.csv', dtype=str)

df_prescriptions.head()

In [ ]:

# ndc_codes = set()

# for index, record in df_prescriptions.iterrows():
#     ndc_codes.add(record['NDC'].strip())
        
# ndc_codes = list(ndc_codes)

# print(len(ndc_codes))

In [ ]:
df = pd.read_csv('../data/processed/NOTE_EVENTS_WITH_DRUGS.csv', dtype=str)

df = df.head(1000)

In [ ]:
df.head()

In [ ]:
df['DRUGS_PRESENT_SPLIT'] = df['DRUGS_PRESENT'].str.split(pat=',') #.apply(lambda x: x.split(','))

df = df[df['DRUGS_PRESENT_SPLIT'].notnull()]

print("Count After Null Filter:", df.count())

In [ ]:

free_text = df['CLEAN_TEXT']
label_codes = df['DRUGS_PRESENT_SPLIT']

### Text Vectorization

In [ ]:

text_vectorizer = TfidfVectorizer(ngram_range=(3, 3),
                             analyzer='char')
X = text_vectorizer.fit_transform(free_text)

In [ ]:
print(X.shape, text_vectorizer.get_feature_names())
print("....", X[0,0])

### Label Encoding

In [ ]:
Y = np.array(label_codes)



In [ ]:
len(set(Y[0]))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
    
lb = MultiLabelBinarizer()
Y = lb.fit_transform(Y)

# Y = Y.T

In [ ]:
print("Labels found in first example: ", Y[0].sum())

print(Y[0])

In [ ]:
print("X: ", X.shape, " Y: ", Y.shape)

In [ ]:
train_samples, no_of_featres = X.shape
ignore, label_count = Y.shape

print(train_samples, no_of_featres, label_count)

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109)

In [ ]:
def collate_fn(data):
    
#     print("collate.....", len(data))
#     print("collate.....", data[0].shape)
    
    data = np.array(data).reshape((len(data), data[0].shape[0], data[0].shape[1]))
    
    res = torch.tensor(data)
    
    print("collate.....", res.dtype, res.shape)
    return res

def load_data(train_dataset, val_dataset):
    
    batch_size = 32
    # your code here
    train_loader = DataLoader(train_dataset.todense(), batch_size = 64, shuffle = True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset.todense(), batch_size = 64, shuffle = False, collate_fn=collate_fn)
    
    return train_loader, val_loader



In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class Network(nn.Module):

    def __init__(self):
        super().__init__()

        self.fc1 = nn.LSTM(no_of_featres, 1000, 1)
        self.a1 = nn.ReLU()
        self.fc2 = nn.Linear(1000, label_count)

    def forward(self,x):
        print(".....", x[0].dtype)
        
        x = self.fc1(x)
        x = self.a1(x)
        x = self.fc2(x)

        return x

In [ ]:
print(torch.cuda.is_available())

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {device} device", device)

from torch.optim import Adam

model = Network()

criterion = nn.MSELoss()
EPOCHS = 10
optimizer = Adam(model.parameters(), lr = 0.001)


In [ ]:

train_loader, val_loader = load_data(X_train, X_test)

In [ ]:
def train(model, train_loader, val_loader, n_epochs):    
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x in train_loader:
            print("Batch..", x.dtype)
            
            loss = None
            
            optimizer.zero_grad()
            y_hat = model(x)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print("Epoch: ", epoch, "Loss: ", train_loss)
#         p, r, f, roc_auc = eval_model(model, val_loader)

In [ ]:
train(model, train_loader, val_loader, 5)

In [ ]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred, average='micro'))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred, average='micro'))

In [ ]:
import pickle
with open('C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\vectorizer.pk', 'wb') as fin:
    pickle.dump(text_vectorizer, fin)

In [ ]:
output = open('C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\label_encoder.pkl', 'wb')
pickle.dump(lb, output)
output.close()

In [ ]:
from joblib import dump, load

dump(model, 'C:\Users\harim\OneDrive\Documents\GitHub\dl4h_final_project_1\trained_models\model.joblib') 